In [1]:
#Exemplo mostrando como eliminar o multiindex de um dataframe importado do Datastream

import DatastreamPy as dsws
import pandas as pd
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt

#Autenticação da API do Datastream
load_dotenv()
ds_user = os.getenv("DSUSER")
ds_pass = os.getenv("DSPWD")

ds = dsws.DataClient(username=ds_user,password=ds_pass)


In [2]:
#Achatar Cabeçalhos
def achatar_cabecalhos(df: pd.DataFrame, sep: str = " | ") -> pd.DataFrame:
    """Um problema comum ao se importar dados do Datastream é que os cabeçalhos dispostos em Multiindex,
    o que ajuda na sua interpretação, mas pode ser prejudicial para se tratar os doados.
    Essa função achata os cabeçalhos em um único nível, facilitando a manipulação dos dados."""

    
    out = df.copy()
    if isinstance(out.columns, pd.MultiIndex):
        out.columns = [sep.join([str(part) for part in col if part is not None and str(part) != ""])
        for col in out.columns]
        return out


In [ ]:
#Carregando os tickers em "códigos" e os Datatypes em "campos"
codigos = "<VALE3.SA>,<PETR4.SA>,<B3SA3.SA>,<ABEV3.SA>,<WEGE3.SA>"
campos = ["MV", "WC01001A", "WC18198A", "WC01705A", "NOSH", "WC08231A", "WC08231A","WC18199A"] 

nomes_campos = {
"MV": "Market Cap",
"WC01001A":"Revenue", 
"WC18198A":"EBITDA", 
"WC01705A":"Net Income",
"NOSH":"Shares Outstanding", 
"WC08231A":"Total Debt % Common Equity",
"WC08231A":"Total Debt % Common Equity",
"WC18199A": "Net Debt"}

inicio = "-10Y" #Ou coloque a data completa no formato YYYY-MM-DD
fim = "0D" #Ou coloque a data completa no formato YYYY-MM-DD
freq = "Q"

#Cria a lista com os nomes dos ativos:
nomes_ativos = ds.get_data(tickers=codigos, fields=["RIC"], kind = 0)

# Cria um dicionário relacionando cada linha de "Instrument" com seu respectivo "Value"
instrument_value_dict = dict(zip(nomes_ativos["Instrument"], nomes_ativos["Value"]))

#Criar o df com os dados do Income Statement e demais indicadores financeiros
FinMetrics = ds.get_data(tickers=codigos, fields=campos, kind = 1, freq = freq, start= inicio, end=fim)

# Substitui os nomes das colunas pelos nomes dos indicadores financeiros e os nomes dos ativos
FinMetrics = FinMetrics.rename(columns=instrument_value_dict)
FinMetrics = FinMetrics.rename(columns=nomes_campos)
#FinMetrics.columns = [nomes_campos.get(col, col) for col in FinMetrics.columns]

FinMetrics


Instrument,VALE3.SA,PETR4.SA,B3SA3.SA,ABEV3.SA,WEGE3.SA,VALE3.SA,PETR4.SA,B3SA3.SA,ABEV3.SA,WEGE3.SA,...,VALE3.SA,PETR4.SA,B3SA3.SA,ABEV3.SA,WEGE3.SA,VALE3.SA,PETR4.SA,B3SA3.SA,ABEV3.SA,WEGE3.SA
Field,Market Cap,Market Cap,Market Cap,Market Cap,Market Cap,Revenue,Revenue,Revenue,Revenue,Revenue,...,Total Debt % Common Equity,Total Debt % Common Equity,Total Debt % Common Equity,Total Debt % Common Equity,Total Debt % Common Equity,Net Debt,Net Debt,Net Debt,Net Debt,Net Debt
Currency,C,C,C,C,C,C,C,C,C,C,...,NA,NA,NA,NA,NA,C,C,C,C,C
Dates,,,,,,,,,,,,,,,,,,,,,
2016-02-06,32911.80,25433.27,18712.66,285117.4,23569.55,22067358.0,70337000.0,563508.0,11565098.0,2416344.0,...,85.65,169.35,11.89,8.41,79.09,98536546.0,364315000.0,-6491194.0,-2628947.0,908633.0
2016-05-06,54306.08,56468.56,29911.20,296119.8,24183.00,23202406.0,71320000.0,574474.0,10377204.0,2335255.0,...,80.81,144.79,10.95,8.20,73.28,88739139.0,327582000.0,-6314243.0,-2663363.0,129178.0
2016-08-06,61029.99,65263.78,34702.80,300206.4,25652.07,23772495.0,70443000.0,559146.0,10482788.0,2238078.0,...,79.18,151.16,10.86,8.87,81.24,84203814.0,319712000.0,-6738857.0,-3561881.0,8727.0
2016-11-06,69523.31,89576.63,31925.86,278516.1,27411.70,25591005.0,70489000.0,623653.0,13177471.0,2377331.0,...,77.82,152.06,28.66,12.04,73.80,84238438.0,308843000.0,-5825306.0,-2959965.0,-379067.0
2017-02-06,96258.13,83806.50,33867.92,268299.8,26120.21,26742046.0,68365000.0,608333.0,11241805.0,2134229.0,...,70.34,139.22,26.52,10.96,72.93,71758422.0,295151000.0,-6737701.0,-2282473.0,-436023.0
2017-05-06,83807.63,79605.00,40050.24,297062.8,29865.53,23362809.0,66996000.0,970903.0,10267953.0,2280769.0,...,68.82,143.88,26.39,10.29,73.29,72693009.0,290834000.0,2383343.0,-4108053.0,-422996.0
2017-08-06,100472.60,75067.31,43365.46,302721.1,30898.71,28600029.0,71822000.0,1060764.0,11362303.0,2435085.0,...,57.31,135.51,23.17,7.06,68.62,66323724.0,274540000.0,1139424.0,-5277502.0,-374612.0


In [4]:
# Testa se o DataFrame é MultiIndex
is_multiindex = isinstance(FinMetrics.columns, pd.MultiIndex)
is_multiindex

# Ao tentar manipular o dataframe como por exemplo para criar uma nova coluna, a interpretação das colunas pode ficar mais complexa.


True

In [5]:
#Usando a função achatar_cabecalhos, a interpretação das colunas fica mais simples.
FinMetrics_achatado = achatar_cabecalhos(FinMetrics)
FinMetrics_achatado

,VALE3.SA | Market Cap | C,PETR4.SA | Market Cap | C,B3SA3.SA | Market Cap | C,ABEV3.SA | Market Cap | C,WEGE3.SA | Market Cap | C,VALE3.SA | Revenue | C,PETR4.SA | Revenue | C,B3SA3.SA | Revenue | C,ABEV3.SA | Revenue | C,WEGE3.SA | Revenue | C,...,VALE3.SA | Total Debt % Common Equity | NA,PETR4.SA | Total Debt % Common Equity | NA,B3SA3.SA | Total Debt % Common Equity | NA,ABEV3.SA | Total Debt % Common Equity | NA,WEGE3.SA | Total Debt % Common Equity | NA,VALE3.SA | Net Debt | C,PETR4.SA | Net Debt | C,B3SA3.SA | Net Debt | C,ABEV3.SA | Net Debt | C,WEGE3.SA | Net Debt | C
Dates,,,,,,,,,,,,,,,,,,,,,
2016-02-06,32911.80,25433.27,18712.66,285117.4,23569.55,22067358.0,70337000.0,563508.0,11565098.0,2416344.0,...,85.65,169.35,11.89,8.41,79.09,98536546.0,364315000.0,-6491194.0,-2628947.0,908633.0
2016-05-06,54306.08,56468.56,29911.20,296119.8,24183.00,23202406.0,71320000.0,574474.0,10377204.0,2335255.0,...,80.81,144.79,10.95,8.20,73.28,88739139.0,327582000.0,-6314243.0,-2663363.0,129178.0
2016-08-06,61029.99,65263.78,34702.80,300206.4,25652.07,23772495.0,70443000.0,559146.0,10482788.0,2238078.0,...,79.18,151.16,10.86,8.87,81.24,84203814.0,319712000.0,-6738857.0,-3561881.0,8727.0
2016-11-06,69523.31,89576.63,31925.86,278516.1,27411.70,25591005.0,70489000.0,623653.0,13177471.0,2377331.0,...,77.82,152.06,28.66,12.04,73.80,84238438.0,308843000.0,-5825306.0,-2959965.0,-379067.0
2017-02-06,96258.13,83806.50,33867.92,268299.8,26120.21,26742046.0,68365000.0,608333.0,11241805.0,2134229.0,...,70.34,139.22,26.52,10.96,72.93,71758422.0,295151000.0,-6737701.0,-2282473.0,-436023.0
2017-05-06,83807.63,79605.00,40050.24,297062.8,29865.53,23362809.0,66996000.0,970903.0,10267953.0,2280769.0,...,68.82,143.88,26.39,10.29,73.29,72693009.0,290834000.0,2383343.0,-4108053.0,-422996.0
2017-08-06,100472.60,75067.31,43365.46,302721.1,30898.71,28600029.0,71822000.0,1060764.0,11362303.0,2435085.0,...,57.31,135.51,23.17,7.06,68.62,66323724.0,274540000.0,1139424.0,-5277502.0,-374612.0
2017-11-06,171615.60,97643.50,48080.89,322839.6,35047.61,29827138.0,76512000.0,1033596.0,15027215.0,2673747.0,...,51.75,136.93,23.19,5.55,60.36,59723734.0,280752000.0,-12660.0,-8163396.0,-529164.0
2018-02-06,224431.60,111984.80,51457.88,339814.7,37420.68,27932425.0,74461000.0,1111922.0,11640219.0,2551476.0,...,45.37,125.38,22.72,9.15,60.72,49305902.0,270712000.0,-1160176.0,-3864558.0,-345623.0


In [6]:
# Testa se o DataFrame é MultiIndex
is_multiindex = isinstance(FinMetrics_achatado.columns, pd.MultiIndex)
is_multiindex

#Com single index, tratar e fazer operações com uma nova coluna é mais simples.

False

In [98]:
#TERMINOU O EXEMPLO DO ACHATAMENTO DE CABEÇALHOS

In [ ]:
# Novo dataframe somente com Net Debt/EBITDA
net_debt_cols = [c for c in FinMetrics_achatado.columns if " | Net Debt" in c]
ebitda_cols = [c for c in FinMetrics_achatado.columns if " | EBITDA" in c]

empresas = sorted({c.split(" | ")[0] for c in net_debt_cols} & {c.split(" | ")[0] for c in ebitda_cols})

df_net_debt_ebitda = pd.DataFrame(index=FinMetrics_achatado.index)
for empresa in empresas:
    net_debt_col = [c for c in net_debt_cols if c.startswith(f"{empresa} |")][0]
    ebitda_col = [c for c in ebitda_cols if c.startswith(f"{empresa} |")][0]
    df_net_debt_ebitda[f"{empresa} | Net Debt/EBITDA"] = (
        FinMetrics_achatado[net_debt_col] / FinMetrics_achatado[ebitda_col]
    )

df_net_debt_ebitda


In [7]:
#transforma a string de codigos em uma lista
codigos_lista = codigos.replace("<", "").replace(">", "").split(",")

In [8]:
# Cria um novo dataframe adicionando a coluna "Debt/Ebtida" para cada empresa

FinMetrics_debt_ebtida = FinMetrics_achatado.copy()
for empresa in codigos_lista:
    net_debt_cols = [c for c in FinMetrics_debt_ebtida.columns if c.startswith(f"{empresa} | Net Debt")]
    ebitda_cols = [c for c in FinMetrics_debt_ebtida.columns if c.startswith(f"{empresa} | EBITDA")]

    if not net_debt_cols or not ebitda_cols:
        raise ValueError(f"Colunas de Net Debt ou EBITDA nao encontradas para {empresa}")

    net_debt_col = net_debt_cols[0]
    ebitda_col = ebitda_cols[0]
    ratio_col = f"{empresa} | Debt/Ebtida"

    FinMetrics_debt_ebtida[ratio_col] = FinMetrics_debt_ebtida[net_debt_col] / FinMetrics_debt_ebtida[ebitda_col]

FinMetrics_debt_ebtida

,VALE3.SA | Market Cap | C,PETR4.SA | Market Cap | C,B3SA3.SA | Market Cap | C,ABEV3.SA | Market Cap | C,WEGE3.SA | Market Cap | C,VALE3.SA | Revenue | C,PETR4.SA | Revenue | C,B3SA3.SA | Revenue | C,ABEV3.SA | Revenue | C,WEGE3.SA | Revenue | C,...,VALE3.SA | Net Debt | C,PETR4.SA | Net Debt | C,B3SA3.SA | Net Debt | C,ABEV3.SA | Net Debt | C,WEGE3.SA | Net Debt | C,VALE3.SA | Debt/Ebtida,PETR4.SA | Debt/Ebtida,B3SA3.SA | Debt/Ebtida,ABEV3.SA | Debt/Ebtida,WEGE3.SA | Debt/Ebtida
Dates,,,,,,,,,,,,,,,,,,,,,
2016-02-06,32911.80,25433.27,18712.66,285117.4,23569.55,22067358.0,70337000.0,563508.0,11565098.0,2416344.0,...,98536546.0,364315000.0,-6491194.0,-2628947.0,908633.0,6.713398,18.904831,-11.062700,-0.594638,1.962139
2016-05-06,54306.08,56468.56,29911.20,296119.8,24183.00,23202406.0,71320000.0,574474.0,10377204.0,2335255.0,...,88739139.0,327582000.0,-6314243.0,-2663363.0,129178.0,6.721323,15.197495,113.424761,-0.734889,0.301751
2016-08-06,61029.99,65263.78,34702.80,300206.4,25652.07,23772495.0,70443000.0,559146.0,10482788.0,2238078.0,...,84203814.0,319712000.0,-6738857.0,-3561881.0,8727.0,10.506199,133.491441,-17.033317,-0.972875,0.018349
2016-11-06,69523.31,89576.63,31925.86,278516.1,27411.70,25591005.0,70489000.0,623653.0,13177471.0,2377331.0,...,84238438.0,308843000.0,-5825306.0,-2959965.0,-379067.0,7.656613,13.471887,-10.004768,-0.441945,-0.723926
2017-02-06,96258.13,83806.50,33867.92,268299.8,26120.21,26742046.0,68365000.0,608333.0,11241805.0,2134229.0,...,71758422.0,295151000.0,-6737701.0,-2282473.0,-436023.0,4.820009,12.029794,-11.406956,-0.596417,-1.014373
2017-05-06,83807.63,79605.00,40050.24,297062.8,29865.53,23362809.0,66996000.0,970903.0,10267953.0,2280769.0,...,72693009.0,290834000.0,2383343.0,-4108053.0,-422996.0,15.224442,12.642208,3.778718,-1.144580,-0.929764
2017-08-06,100472.60,75067.31,43365.46,302721.1,30898.71,28600029.0,71822000.0,1060764.0,11362303.0,2435085.0,...,66323724.0,274540000.0,1139424.0,-5277502.0,-374612.0,4.346480,15.814516,1.378360,-1.261258,-0.770310
2017-11-06,171615.60,97643.50,48080.89,322839.6,35047.61,29827138.0,76512000.0,1033596.0,15027215.0,2673747.0,...,59723734.0,280752000.0,-12660.0,-8163396.0,-529164.0,7.744662,37.827001,-0.016786,-1.277943,-1.201111
2018-02-06,224431.60,111984.80,51457.88,339814.7,37420.68,27932425.0,74461000.0,1111922.0,11640219.0,2551476.0,...,49305902.0,270712000.0,-1160176.0,-3864558.0,-345623.0,4.133904,9.440697,-1.428377,-0.876007,-0.743781


In [ ]:
# Cria um novo DataFrame somente com as colunas "Debt/Ebtida" de cada empresa

# Encontrar todas as colunas que representam Debt/Ebtida

In [ ]:
# Cria um novo dataframe adicionando a coluna "Debt/Ebtida" para cada empresa

codigos_lista = codigos.replace("<", "").replace(">", "").split(",")

FinMetrics_debt_ebtida = FinMetrics.copy()
for empresa in codigos_lista:
    net_debt_cols = [c for c in FinMetrics_debt_ebtida.columns if c.startswith(f"{empresa} | Net Debt")]
    ebitda_cols = [c for c in FinMetrics_debt_ebtida.columns if c.startswith(f"{empresa} | EBITDA")]

    if not net_debt_cols or not ebitda_cols:
        raise ValueError(f"Colunas de Net Debt ou EBITDA nao encontradas para {empresa}")

    net_debt_col = net_debt_cols[0]
    ebitda_col = ebitda_cols[0]
    ratio_col = f"{empresa} | Net Debt/Ebtida"

    FinMetrics_debt_ebtida[ratio_col] = FinMetrics_debt_ebtida[net_debt_col] / FinMetrics_debt_ebtida[ebitda_col]

FinMetrics_debt_ebtida

AttributeError: 'tuple' object has no attribute 'startswith'

In [73]:
codigos = "<VALE3.SA>,<PETR4.SA>,<B3SA3.SA>,<ABEV3.SA>,<BRFS3.SA>,<CPLE6.SA>,<CSNA3.SA>,<ELET3.SA>,<GGBR4.SA>,<HYPE3.SA>,<ITUB4.SA>,<JBSS3.SA>,<LREN3.SA>,<MGLU3.SA>,<MRFG3.SA>,<PCAR3.SA>,<PETR3.SA>,<RADL3.SA>,<RENT3.SA>,<SANB11.SA>,<SBSP3.SA>,<SUZB3.SA>,<TIMS3.SA>,<TOTS3.SA>,<VIVT4.SA>,<WEGE3.SA>"

In [75]:
codigos_lista

['VALE3.SA',
 'PETR4.SA',
 'B3SA3.SA',
 'ABEV3.SA',
 'BRFS3.SA',
 'CPLE6.SA',
 'CSNA3.SA',
 'ELET3.SA',
 'GGBR4.SA',
 'HYPE3.SA',
 'ITUB4.SA',
 'JBSS3.SA',
 'LREN3.SA',
 'MGLU3.SA',
 'MRFG3.SA',
 'PCAR3.SA',
 'PETR3.SA',
 'RADL3.SA',
 'RENT3.SA',
 'SANB11.SA',
 'SBSP3.SA',
 'SUZB3.SA',
 'TIMS3.SA',
 'TOTS3.SA',
 'VIVT4.SA',
 'WEGE3.SA']